# Grouping scans by fieldmaps and phase encoding directions



In [1]:
%%bash

SAMPLE_URL="https://upenn.box.com/shared/static/bvhs3sw2swdkdyekpjhnrhvz89x3k87t.xz"
wget --retry-connrefused --waitretry=5 --read-timeout=20 --timeout=15 -t 0 -q \
 -O dscsdsi_buds.tar.xz ${SAMPLE_URL}
 
tar xvfJ dscsdsi_buds.tar.xz
ls DSCSDSI_BUDS
rm dscsdsi_buds.tar.xz



README
dataset_description.json
sub-tester


x DSCSDSI_BUDS/
x DSCSDSI_BUDS/sub-tester/
x DSCSDSI_BUDS/sub-tester/fmap/
x DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-AP_epi.json
x DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-AP_epi.nii.gz
x DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-PA_epi.json
x DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-PA_epi.nii.gz
x DSCSDSI_BUDS/sub-tester/dwi/
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.bval
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.json
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.bvec
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.bval
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.nii.gz
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.nii.gz
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.bvec
x DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.json
x DSCSDSI_BUDS/sub-tester/anat/
x DSCSDSI_BUDS/sub-tester/anat/sub-tester_T1w.nii.gz
x DSCSDSI_BUDS/sub-tester/anat/sub-tester_T1w.json
x DSCSDSI_BUDS

In [2]:
from qsiprep.utils.bids import collect_data

def get_session_groups(layout, subject_data, combine_all_dwis):
    # Handle the grouping of multiple dwi files within a session
    sessions = layout.get_sessions() if layout is not None else []
    all_dwis = subject_data['dwi']
    dwi_session_groups = []
    if not combine_all_dwis:
        dwi_session_groups = [[dwi] for dwi in all_dwis]
    else:
        if sessions:
            print('Combining all dwi files within each available session:')
            for session in sessions:
                session_files = [img for img in all_dwis if 'ses-'+session in img]
                print('\t- %d sessions in %s', len(session_files), session)
                dwi_session_groups.append(session_files)
        else:
            print(
                'Combining all %d dwis within the single available session' % len(all_dwis))
            dwi_session_groups = [all_dwis]
    return dwi_session_groups

subject_data, layout = collect_data("DSCSDSI_BUDS", "tester")
print(layout)

# Handle the grouping of multiple dwi files within a session
dwi_session_groups = get_session_groups(layout, subject_data, combine_all_dwis=True)
print(dwi_session_groups)

BIDS Layout: ...qsiprep/notebooks/DSCSDSI_BUDS | Subjects: 1 | Sessions: 0 | Runs: 0
Combining all 2 dwis within the single available session
[['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.nii.gz', '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.nii.gz']]


This is a list of both files because combine_all_dwis is true. How these are combined depends on whether they should be combined in a blip-up/blip-down series (BUDS) or whether they should be corrected using their fieldmaps.

```python
for dwi_session_group in dwi_session_groups:
    dwi_fmap_groups.extend(
        group_by_fieldmaps(dwi_session_group, layout,
                           "fieldmaps" in ignore or force_syn,
                           prefer_dedicated_fmaps,
                           combine_all_dwis))
```

Instead we will walk though the `group_by_fieldmaps` function to show how it works

In [7]:
from collections import defaultdict

# There is only one group so don't loop
dwi_files = dwi_session_groups[0]

def group_by_fieldmaps(dwi_files, layout, ignore_fieldmap, prefer_dedicated_fmaps,
                       combine_all_dwis):
    """Groups a session's DWI files by their PE direction and fieldmaps, if specified.
    
    DWIs are grouped by their **warped space**. Two DWI series that are 
    listed in the IntendedFor field of a fieldmap are assumed to have the same 
    susceptibility distortions and therefore be in the same warped space. The goal 
    of this function is to combine DWI series into groups of acquisitions that 
    are in the same warped space into a list of scans that can be combined after
    unwarping. 
    
    The first step in the pipeline must be ``dwidenoise``. This algorithm can 
    be run directly on each DWI Series. OR, DWI series in the same warped space 
    can be combined before running ``dwidenoise``.  Running ``dwidenoise`` on 
    concatenated DWI series that include noise and susceptibility distortion 
    violates the assumptions made by ``dwidenoise`` and is not allowed. 
    Performing susceptibility distortion correction before running ``dwidenoise``
    is also not allowed.
    
    Similarly, SHORELine performs head motion correction in **warped space**.
    The more q-space samples are available to SHORELine, the better the EAP
    estimates. However, if the position of the head is very different between
    the two runs it may take more iterations for SHORELine to converge. Be sure 
    to check the QC files if using this algorithm.
    
    Specifying ``combine_all_dwis=True`` means that the SDC outputs will be 
    used to combine across warped spaces. If ``False``, then no scans will be 
    combined even if they are in the same warped space. Other 
    possibilities for grouping are not supported.
    

    Parameters
    -----------
    
        dwi_files: list
            A list of full paths to dwi nifti files in a BIDS tree
        
        layout: BIDSLayout
            A representation of the BIDS tree
            
        prefer_dedicated_fmaps: bool
            If True, use files in fmap directories to perform SDC even if there 
            is a series in dwi with the reverse phase encoding direction that 
            could be used.
            
        combine_all_dwis: bool
            If True, concaenate DWI series that can be concatenated. To be concatenated
            the series must either have the same phase encoding direction or there 
            must be fieldmaps that correct for opposite phase encoding directions.

    Returns:
    ---------
    
        groups_list: list
            A list of dictionaries that describe "group info". These can take a couple forms
            
            When there are two groups of dwi series with opposite phase encoding dirs:
            [
                {'dwi_series': ['scan1.nii', 'scan2.nii'], 
                 'rpe_dwi_series': ['scan3.nii', 'scan4.nii'],
                 'rpe_b0': [],
                 'dwi_series_pedir': 'j'}
            ]
            
            When there is a PEPolar fieldmap
            [
             {
              'dwi_series': ['scan1.nii', 'scan2.nii'],
              'rpe_dwi_series': [],
              'rpe_b0: ['fmap_scan.nii'],
              'dwi_series_pedir': 'j'
             }
            ]
            
            When two different PE directions each have their own fieldmaps
            [
             {
              'dwi_series': ['scan1.nii', 'scan2.nii'],
              'rpe_dwi_series': [],
              'rpe_b0: ['fmap_scan.nii'],
              'dwi_series_pedir': 'j'
             },
             {
              'dwi_series': ['scan3.nii', 'scan4.nii'],
              'rpe_dwi_series': [],
              'rpe_b0: ['fmap_scan2.nii'],
              'dwi_series_pedir': 'j-'
             }
            ]
            
        NOTE: the pre-hmc steps, such as denoising, will only be run within warped space
        groups 
            
            
        
    """

    # For doc-building
    if layout is None:
        return [{'dwi_series':dwi_series,
                 'rpe_series':[],
                 'rpe_b0':[],
                 'dwi_series_pedir': 'j'}]

    # For each DWI, figure out its PE dir and whether or not it was listed in
    # an IntendedFor field in a fieldmap sidecar. Make a list of
    # (filename, PE dir, fmap_file)
    parsed_dwis = []
    for ref_file in dwi_files:
        metadata = layout.get_metadata(ref_file)

        # Find fieldmaps. Options: (epi only)
        fmaps = []
        fmap_file = ''

        fmaps = layout.get_fieldmap(ref_file, return_list=True)
        fmap_files = []
        for fmap in fmaps:
            if fmap['type'] == 'epi':
                fmap_files.append(fmap['epi'])

        num_fmaps = len(fmap_files)
        if num_fmaps > 1:
            print("Multiple field maps found for %s", ref_file)
        if num_fmaps >= 1:
            fmap_file = fmap_files[0]
        parsed_dwis.append(
            (ref_file, metadata['PhaseEncodingDirection'], fmap_file))
    
    # If we're not combining, each DWI series is its own group
    groups = []
    if not combine_all_dwis:
        for ref_file, pe_dir, fmap_file in parsed_dwis:
            groups.append({
                'dwi_series': [ref_file],
                'rpe_series': [],
                'rpe_b0': fmap_file,
                'dwi_series_pedir': pe_dir
            })
        return groups

    # group by warped space, defined by pe_dir and fmap
    groups = defaultdict(list)
    for ref_file, pe_dir, fmap in parsed_dwis:
        groups[(pe_dir, fmap)].append(ref_file)
        

    # Which groups can be combined to do bidirectional pepolar?
    complete_groups = []
    group_keys = set(groups.keys())

    while len(group_keys):
        
        key = group_keys.pop()
        scan_list = groups[key]
        pe_dir, fmap = key
        pe_axis = pe_dir[0]
        
        if prefer_dedicated_fmaps:
            matches = [k for k in group_keys if k[0][0] == pe_axis and fmap == k[1]]
        else:
            matches = [k for k in group_keys if k[0][0] == pe_axis]
        num_matches = len(matches)
        
        if not num_matches == 1:
            # Didn't find any reverse PE groups
            #print("No matches found for %s" % key)
            complete_groups.append(
                {'dwi_series': scan_list,
                 'rpe_series': [],
                 'rpe_b0': fmap,
                 'dwi_series_pedir': pe_dir
                })
        else:
            # Found a perfect match
            match = matches[0]
            match_scan_list = groups[match]
            complete_groups.append( 
                {'dwi_series': scan_list,
                 'rpe_series': match_scan_list,
                 'rpe_b0': [],
                 'dwi_series_pedir': pe_dir}
            )
            group_keys.discard(match)
            print("Matched %s to %s" % (match, key))

    return complete_groups


Suppose I want to use the opposite phase-encoding DWI series to correct for susceptibility. 

In [8]:
ignore_fieldmap=False
prefer_dedicated_fmaps=False
combine_all_dwis=True

group_by_fieldmaps(dwi_files, layout, ignore_fieldmap, prefer_dedicated_fmaps,
                   combine_all_dwis)

Matched ('j-', '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-AP_epi.nii.gz') to ('j', '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-PA_epi.nii.gz')


[{'dwi_series': ['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.nii.gz'],
  'rpe_series': ['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.nii.gz'],
  'rpe_b0': [],
  'dwi_series_pedir': 'j'}]

If you prefer to use the files in the fmap directory, 

In [9]:
ignore_fieldmap=False
prefer_dedicated_fmaps=True
combine_all_dwis=True

group_by_fieldmaps(dwi_files, layout, ignore_fieldmap, prefer_dedicated_fmaps,
                   combine_all_dwis)

[{'dwi_series': ['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.nii.gz'],
  'rpe_series': [],
  'rpe_b0': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-PA_epi.nii.gz',
  'dwi_series_pedir': 'j'},
 {'dwi_series': ['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.nii.gz'],
  'rpe_series': [],
  'rpe_b0': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-AP_epi.nii.gz',
  'dwi_series_pedir': 'j-'}]

In [10]:
ignore_fieldmap=False
prefer_dedicated_fmaps=True
combine_all_dwis=False

group_by_fieldmaps(dwi_files, layout, ignore_fieldmap, prefer_dedicated_fmaps,
                   combine_all_dwis)

[{'dwi_series': ['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55AP_dwi.nii.gz'],
  'rpe_series': [],
  'rpe_b0': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-PA_epi.nii.gz',
  'dwi_series_pedir': 'j'},
 {'dwi_series': ['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/dwi/sub-tester_acq-HASC55PA_dwi.nii.gz'],
  'rpe_series': [],
  'rpe_b0': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_BUDS/sub-tester/fmap/sub-tester_dir-AP_epi.nii.gz',
  'dwi_series_pedir': 'j-'}]